In [ ]:
import os
import pandas as pd
import metrics as mt
import os
import json
from rdkit import Chem
import numpy as np
import torch.nn as nn
from captum.attr import IntegratedGradients
from dgllife.utils import CanonicalAtomFeaturizer
from explainers_callers import (get_model_and_featurizer, get_ig_scores, get_subgX_scores,
                                get_molgraphx_scores, DEVICE)   
from Source.explainers import visualize
from tqdm import tqdm
from multiprocessing import Pool
from tqdm.contrib.concurrent import process_map
from torch import Tensor, nan, nan_to_num
from Source.explainers.subgraphX.utils import calculate_atom_scores_multiple_coalitions

dataset_name = "N_O"
MODEL_FOLDER = f"/home/cairne/WorkSpace/molgraphX_paper_scripts/GCNN_2D/Output/ibench_{dataset_name}"
PATH_TO_SDF = f"/home/cairne/WorkSpace/molgraphX_paper_scripts/Data/ibenchmark/Datasets/{dataset_name}_test_lbl.sdf"
molecules = [mol for mol in Chem.SDMolSupplier(PATH_TO_SDF) if mol is not None]
molecules.sort(key=lambda mol: mol.GetNumAtoms())
model, featurizer = get_model_and_featurizer(MODEL_FOLDER)


In [ ]:
from rdkit.Chem import Descriptors
from time import time
import pandas as pd
from tqdm import tqdm
results = {"num_a": [], "molw": [], "subgx": [], "ig": [], "molgx_sm": [], "molgx_bi": []}

for mol in tqdm(molecules[::2][:50]):
    results["molw"].append(Descriptors.ExactMolWt(mol))
    results["num_a"].append(mol.GetNumAtoms())

    
    t = time()
    for i in range(10):
        get_ig_scores(mol, model, featurizer)
    results["ig"].append((time() - t) / 10)

    t = time()
    for i in range(10):
        get_subgX_scores(mol, model, featurizer)
    results["subgx"].append((time() - t) / 10)

    t = time()
    for i in range(10):
        get_molgraphx_scores(mol, model, featurizer, min_atoms=1)
    results["molgx_sm"].append((time() - t) / 10)

    t = time()
    for i in range(10):
        get_molgraphx_scores(mol, model, featurizer, min_atoms=10)
    results["molgx_bi"].append((time() - t) / 10)

df = pd.DataFrame.from_dict(results)
df.to_csv("/home/cairne/WorkSpace/molgraphX_paper_scripts/GCNN_2D/Results/speed.csv")

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# Read the CSV file
# df = pd.read_csv('speed.csv')

# Columns to plot
y_columns = ['subgx', 'ig', 'molgx_sm']
x_column = 'num_a'
legend_labels = {
    'subgx': 'SubgraphX',
    'ig': 'Integrated Gradient',
    'molgx_sm': 'MolgraphX',
}
# Calculate mean and standard deviation for each num_a group
grouped = df.groupby(x_column)[y_columns].agg(['mean', 'std']).reset_index()

# Create the plot
fig = go.Figure()

# Colors for different curves
colors = px.colors.qualitative.Set1

# Add traces for each y column
for i, col in enumerate(y_columns):
    # Get mean values
    y_mean = grouped[(col, 'mean')]
    
    # Get standard deviation for error bars
    y_std = grouped[(col, 'std')]
    
    # Add trace with error bars
    fig.add_trace(go.Scatter(
        x=grouped[x_column],
        y=y_mean,
        mode='lines+markers',
        name=legend_labels[col],
        line=dict(color=colors[i % len(colors)]),
        marker=dict(size=8),
        error_y=dict(
            type='data',
            array=y_std / 2,
            visible=True,
            color=colors[i % len(colors)],
            thickness=1.5,
            width=3
        )
    ))

# Update layout
fig.update_layout(
    # title='Molecular Properties vs Number of Atoms',
    xaxis_title='Number of atoms',
    yaxis_title='Computational time, s',
    template='plotly_white',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ),
    font=dict(  
    # family="Courier New, monospace",  
    size=20,  
    # color="RebeccaPurple"  
    ),
    width=800,
    height=600
)

# Show the plot
fig.show()
fig.write_image("molecular_properties_plot.png")
# Optional: Save the plot as HTML
# fig.write_html("molecular_properties_plot.html")